In [99]:
import networkx as nx
import numpy as np
import pandas as pd

In [100]:
df = pd.read_csv('./full_data.csv')
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Host', 'Parasite', 'ParasiteFull',
       'group', 'locality', 'hostgroup', 'animal_nodes', 'parasites_nodes'],
      dtype='object')

Below is the ready code for the projection without lossing any metadata.

In [101]:
G = nx.Graph()

for i, el in df.iterrows():
    if G.has_node(el['Parasite']):
        G.add_node(el['Parasite'], bipartite=0)
    if G.has_node(el['Host']):
        G.nodes[el['Host']]['locality'].add(el['locality'])
        G.nodes[el['Host']]['para_group'].add(el['group'])
    else:
        locality = set()
        locality.add(el['locality'])
        para_group = set()
        para_group.add(el['group'])
        G.add_node(el['Host'], bipartite=1, host_group=el['hostgroup'], para_group=para_group, locality=locality)
    if not G.has_edge(el['Host'], el['Parasite']):
        G.add_edge(el['Host'], el['Parasite'], )
    

In [102]:
# average resource allocation bipartite projection for host nodes
# ProbS you multiply the stochastic with a stochastic version of the transpose. then get (wu,v + wv,u)/2
# code below:

bipartite_one_nodes = [ n for n, d in G.nodes(data=True) if d.get('bipartite') == 1]
print(len(bipartite_one_nodes))
badj = nx.bipartite.biadjacency_matrix(G, row_order=sorted(bipartite_one_nodes)).toarray()

5870


In [103]:
# stochastic adj matrix
A = badj / badj.sum(axis=1)[:, np.newaxis]
At = badj.T / badj.T.sum(axis=1)[:, np.newaxis]
weights = A @ At

In [104]:
# now we have directional weights, we need to get the average
resource_allocation_weights = (weights + weights.T) / 2


In [105]:
np.allclose(resource_allocation_weights, resource_allocation_weights.T)

True

In [106]:
# compute the number of zero entries
print(np.count_nonzero(resource_allocation_weights == 0))

34006684


In [107]:
mean_weight[:, np.newaxis].shape

(5870, 1)

SELF LOOPS, are we care about them? not much tbh, we can remove them later.

In [108]:
# do the noise corrected disparity filter
# for each
mean_weight = resource_allocation_weights.mean(axis=1)
disparity_filter = (mean_weight[:, np.newaxis] + mean_weight[np.newaxis, :]) / 2

# where the values are below the disparity filter, set to zero
filtered_weights = resource_allocation_weights.copy()
filtered_weights[resource_allocation_weights < disparity_filter] = 0
filtered_weights

array([[0.67559975, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.08333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.07692308, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333333,
        0.33333333],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333333,
        0.33333333]])

In [109]:
# compute the number of zero entries
print(np.count_nonzero(filtered_weights == 0))
filtered_weights.shape

34030846


(5870, 5870)